In [3]:
import json 
import sys
sys.path.append('../')
prefixes = ['B','A0','A1','A2','A3','A4','A5','A6','A7','A8','A9']

In [2]:
def get_words(data,type): 
    lemmata = {}
    tokens = {}
    for item in data.values(): 
        if type != "margin": 
            item = item.items()
        for i in item: 
            encodings = i[1]
            for token, pos, lemma in encodings:
                token = token.strip(".")
                pos = pos.strip(".")
                lemma = lemma.strip(".")
                # proper nouns, abbreviations, or capitalized words 
                if token == pos: continue # no punctuation 
                if len(token) == 0: continue
                if token not in tokens: tokens[token] = 0
                tokens[token] += 1
                if len(pos) == 0: continue 
                if "np" in pos: 
                    if lemma not in lemmata: lemmata[lemma] = 0
                    lemmata[lemma] += 1 
                elif token[0].isupper():
                    if ("n"==pos[0]) or ("j" == pos[0]) or ('fw' in pos) or ('v' in pos):  
                        if token not in lemmata: lemmata[token] = 0
                        lemmata[token] += 1 
    return lemmata, tokens

def add_to_dict(old,new): 
    for word, freq in new.items(): 
        if word not in old: old[word] = freq
        else: old[word] += freq
    return old

In [ ]:
lemmata = {}
tokens = {}
for prefix in prefixes: 
    with open(f"../assets/processed/{prefix}_marginalia.json","r") as file: 
        data = json.load(file)
    l, t = get_words(data,"margin")
    lemmata = add_to_dict(lemmata,l)
    tokens = add_to_dict(tokens, t)
    print(prefix)
        

In [4]:
print(len(tokens), 'types')
print(sum(tokens.values()), 'tokens')
print(len(lemmata),'possible proper nouns')
lemmata = sorted(lemmata.items(), key=lambda x:x[1], reverse=True)
tokens = sorted(tokens.items(), key=lambda x:x[1], reverse=True)
lemmata[:20]

170404 types
2464705 tokens
58335 possible proper nouns


[('Psal', 24247),
 ('Cor', 17266),
 ('Rom', 13450),
 ('Mat', 11644),
 ('Gen', 10244),
 ('John', 8129),
 ('Heb', 7853),
 ('Sam', 7533),
 ('Pet', 6718),
 ('Tim', 6571),
 ('God', 6535),
 ('Act', 5830),
 ('Matth', 5242),
 ('Isa', 5208),
 ('Luk', 5143),
 ('Luke', 5014),
 ('Exod', 4861),
 ('Aug', 4659),
 ('Christ', 4652),
 ('Ioh', 4615)]

In [5]:
with open("../assets/vocab/marginal_word_types.json","w+") as file: 
    json.dump(tokens, file)
with open("../assets/vocab/marginal_proper_nouns.json","w+") as file: 
    json.dump(lemmata, file)

In [6]:
text_lemmata = {}
text_tokens = {}
def get_group_vocab(prefix,text_tokens,text_lemmata): 
    with open(f"../assets/processed/{prefix}_texts.json","r") as file: 
        data = json.load(file)
    l, t = get_words(data,"text")
    text_lemmata = add_to_dict(text_lemmata,l)
    text_tokens = add_to_dict(text_tokens, t)
    print(prefix)
    return text_tokens,text_lemmata

def save(text_tokens, text_lemmata):
    with open("../assets/vocab/text_word_types.json","w+") as file: 
        json.dump(text_tokens, file)
    with open("../assets/vocab/text_proper_nouns.json","w+") as file: 
        json.dump(text_lemmata, file)
    print('saved!')

In [ ]:
for prefix in prefixes: 
    text_tokens, text_lemmata = get_group_vocab(prefix,text_tokens, text_lemmata)
    save(text_tokens, text_lemmata)

In [8]:
print(len(text_tokens), 'types')
print(sum(text_tokens.values()), 'tokens')
print(len(text_lemmata),'possible proper nouns')
text_lemmata = sorted(text_lemmata.items(), key=lambda x:x[1], reverse=True)
text_tokens = sorted(text_tokens.items(), key=lambda x:x[1], reverse=True)
save(text_tokens, text_lemmata)
text_lemmata[:20]


597004 types
109755841 tokens
185631 possible proper nouns


saved!


[('God', 1031383),
 ('Christ', 404056),
 ('Lord', 230838),
 ('Church', 102753),
 ('So', 83613),
 ('Jesus', 81429),
 ('Spirit', 69577),
 ('Now', 64286),
 ('Paul', 58750),
 ('World', 56951),
 ('Father', 56335),
 ('David', 56099),
 ('Apostle', 56000),
 ('King', 54175),
 ('Law', 53038),
 ('Faith', 52938),
 ('Heaven', 52614),
 ('Religion', 52358),
 ('Gods', 49195),
 ('Let', 48134)]

In [4]:
import json 
with open("../assets/vocab/marginal_proper_nouns.json","r") as file: 
    lemmata = json.load(file)
with open("../assets/vocab/text_proper_nouns.json","r") as file: 
    text_lemmata = json.load(file)

In [5]:
words = {x[0]:x[1] for x in lemmata}
text_words = {x[0]:x[1] for x in text_lemmata}

In [86]:
import re,pprint
author_ids = {}

with open("../assets/authors.csv","r",encoding="utf-8") as file: 
    data = file.readlines()

unicode_pattern = re.compile(r'[^\x00-\x7F]+')
for l, line in enumerate(data):
    if l == 0: continue 
    a_id, name_type, name = line.split('","')
    a_id = a_id.split("authors/")[-1].strip('\"')
    name = re.sub(r'\"|\[|\]','',name)
    orig_name = name.strip("\n") 
    orig_name = unicode_pattern.sub('', orig_name)
    name = orig_name.split(" ")
    if a_id not in author_ids: 
        author_ids[a_id] = []
    if len(name) < 5: 
        if name_type in ['lat','default','eng','abbr']:
            # if len(name) == 3 and name_type == "eng": 
            #     name = name[1]
            # elif len(name) == 2 and name_type == "eng": 
            #     if len(name[0].split(" ")) == 1: 
            #         name = name[1]
            #     else: 
            #         name = name[0]
            author_ids[a_id].append(orig_name)
            

In [122]:
with open(f"../assets/TIPNR - Translators Individualised Proper Names with all References - STEPBible.org CC BY.txt") as file: 
    data = file.readlines()

In [139]:
in_persons_section = False
entities = []
for idx, line in enumerate(data): 
    line = line.strip()
    if not line:
        continue
    
    if line.startswith('$========== PERSON(s)'):
        in_persons_section = True
        continue  
    
    if idx < 112: continue
    if idx > 14456: continue
    if in_persons_section:
        name = line.split('=')[0]
        name_parts = name.split('@')
        if len(name_parts) > 1: 
            person_name = name_parts[0].strip() 
            if "–" in person_name or "ADDED" in person_name or "(" in person_name or "-" in person_name: 
                person_name = person_name.split("	")[-1]

            if person_name[0].islower() or "–" in person_name[0]: continue
            if "|" in person_name: 
                entities.extend(person_name.split("|"))
            else: 
                entities.append(person_name)
    else:
        continue
entities = list(sorted(set(entities)))

In [171]:
prefixes = {}
for word in words.keys(): 
    if word[:1] not in prefixes: 
        prefixes[word[:1]] = []
    prefixes[word[:1]].append(word)

In [172]:
m_hits = {}
for name in entities:
    if name in words: m_hits[name] = words[name]

In [173]:
len(entities)

3157

In [174]:
len(m_hits)

599

In [175]:
text_prefixes = {}
for word in text_words: 
    if word[:1] not in text_prefixes: 
        text_prefixes[word[:1]] = []
    text_prefixes[word[:1]].append(word)

In [178]:
t_hits = {}
for name in entities:
    if name in text_words: t_hits[name] = text_words[name]

In [179]:
len(t_hits)

1559

In [199]:
a_hits_margin = {}
for auts in author_ids.values(): 
    for aut in auts: 
        aut = aut.split(" ")
        for a in aut: 
            if len(a.strip(".")) < 3: continue
            if a in words: 
                a_hits_margin[a] = words[a]

print(len(a_hits_margin))
list(sorted(a_hits_margin.items(),key=lambda x:x[1],reverse=True))[:10]

750


[('John', 8129),
 ('Mark', 1361),
 ('Basil', 653),
 ('Seneca', 396),
 ('Peter', 392),
 ('Iii', 353),
 ('Ecclesiae', 302),
 ('Cicero', 283),
 ('Martyr', 279),
 ('Plato', 273)]

In [195]:
print(len(author_ids))

1401


In [200]:
a_hits_text = {}
for auts in author_ids.values(): 
    for aut in auts: 
        aut = aut.split(" ")
        for a in aut: 
            if len(a.strip(".")) < 3: continue
            if a in text_words: 
                a_hits_text[a] = text_words[a]
print(len(a_hits_text))
list(sorted(a_hits_text.items(),key=lambda x:x[1],reverse=True))[:10]

988


[('John', 31957),
 ('Peter', 24542),
 ('Jerusalem', 18473),
 ('Mark', 5210),
 ('Isaac', 4877),
 ('Caesar', 4032),
 ('War', 3777),
 ('Iii', 2969),
 ('Ambrose', 2895),
 ('Gregory', 2762)]

In [181]:
with open('../assets/bible_hits.json','w+') as file: 
    json.dump({'marginal':m_hits, 'in-text':t_hits},file)

In [202]:
with open('../assets/author_hits.json','w+') as file: 
    json.dump({'marginal':a_hits_margin, 'in-text':a_hits_text},file)

Get the most similar words by edit distance to the canonical and biblical hits 

In [20]:
from fuzzywuzzy import process
from Levenshtein import distance 
def dist_fn(s1, s2):
    return distance(s1, s2)

def find_similar_words(target_word, word_list, threshold,k=20):
    similar_words = process.extract(target_word, word_list, limit=None,scorer=dist_fn)
    similar_words = [(word, dist) for word, dist in similar_words if dist <= threshold][-k:]
    similar_words = sorted(similar_words, key=lambda x:x[1])
    return similar_words

def find_match(target_word, word_list):
    match = process.extract(target_word, word_list, limit=None,scorer=dist_fn)
    match = [(word, dist) for word, dist in match if dist == 0]
    if len(match) > 0: 
        return True 
    return False

In [18]:
with open('../assets/author_hits.json','r') as file: 
    a_hits = json.load(file)

with open('../assets/bible_hits.json','r') as file: 
    b_hits = json.load(file)

a_hits = a_hits['marginal']
b_hits = b_hits['marginal']
len(a_hits),len(b_hits)

(750, 599)

In [23]:
def similar_spelling(hits):  
    for target in hits: 
        similar_words = find_similar_words(target, text_words.keys(),len(target)/2,10)
        print(f"Words similar to '{target}':")
        for word, dist in similar_words:
            print(f"{word} (Distance: {dist})")
        print()
similar_spelling(b_hits)

Words similar to 'Aaron':
Aaron (Distance: 0)
Aaron^ (Distance: 0)
Aanon (Distance: 1)
Akron (Distance: 1)
Aaros (Distance: 1)
Aarona (Distance: 1)
aarons (Distance: 1)
Haron (Distance: 1)
aron (Distance: 1)
Adron^ (Distance: 1)

Words similar to 'Abba':
Abba (Distance: 0)
Ab^a (Distance: 1)
Sabba^^ (Distance: 1)
Abea (Distance: 1)
Ahba (Distance: 1)
Abya (Distance: 1)
Sabba^ (Distance: 1)
Absa^^ (Distance: 1)
ABBY (Distance: 1)
Abbah (Distance: 1)

Words similar to 'Abdi':
Abdi (Distance: 0)
Audi (Distance: 1)
Abi (Distance: 1)
Zabdi (Distance: 1)
Abda (Distance: 1)
Abdie (Distance: 1)
Abdia (Distance: 1)
Abdy (Distance: 1)
AIDI (Distance: 1)
A^di (Distance: 1)

Words similar to 'Abednego':
Abednego (Distance: 0)
Abednago (Distance: 1)
Abed-nego (Distance: 1)
Abednigo (Distance: 1)
Abeanego (Distance: 1)
Abedn^go (Distance: 1)
Habednego (Distance: 1)
ABEDN^GO (Distance: 1)
Abedneg (Distance: 1)
Ahedneg^ (Distance: 2)

Words similar to 'Abel':
Abel (Distance: 0)
Abel^ (Distance: 0)
Jab

KeyboardInterrupt: 